In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

/Users/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import json
import string

r = open('../results/fixed_results.json', 'r').read()
e = json.loads(r)

caps_list = []
for meme in e:
    this_meme = []
    try:
        caption, classes, title = meme
    except:
        classes, title = meme
    this_meme.extend([classes[0]])
    if len(caption) is 0 and len(title) is 0:
        break
    elif len(caption) < len(title):
        tokens = title.split()
    else: 
        tokens = caption.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    this_meme.extend(tokens)
    caps_list.append(' '.join(this_meme))

In [7]:
r2 = open('../results/fixed_results_dankmemes.json', 'r').read()
e2 = json.loads(r2)

for meme in e2:
    try:
        this_meme = []
        try:
            caption, classes, title = meme
        except:
            classes, title = meme
        this_meme.extend([classes[0]])
        if len(caption) is 0 and len(title) is 0:
            break
        elif len(caption) < len(title):
            tokens = title.split()
        else: 
            tokens = caption.split()
        # remove punctuation from each token
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]
        # make lower case
        tokens = [word.lower() for word in tokens]
        this_meme.extend(tokens)
        caps_list.append(' '.join(this_meme))
    except:
        continue

In [23]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read().split('\n')
    file.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
# organize into sequences of 10 tokens
sequences = list()
for caption in caps_list:
    cs = caption.split()
    len_cs = len(cs)
    if len_cs > 10:
        cs = cs[:10]
        len_cs = 10
    cs.extend(['PADDING']*9)
    for i in range(len_cs):
        # select sequence of tokens
        seq = cs[i:i+10]
        # convert into a line for printing to the text file
        line = ' '.join(seq)
        # store
        sequences.append(line)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 2231


In [12]:
out_filename = 'padded_captions.txt'
save_doc(sequences, out_filename)

In [24]:
from keras.utils.data_utils import get_file

path = get_file('padded_captions.txt', origin='https://raw.githubusercontent.com/casassg/meme_puller/master/LangModel/padded_captions.txt')
sequences = load_doc(path)

In [25]:
#convert back to lists of tokens for the tokenizer
lines = []
for seq in sequences:
    lines.append(seq.split())

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)

X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 9, 50)             58950     
_________________________________________________________________
lstm_3 (LSTM)                (None, 9, 100)            60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1179)              119079    
Total params: 328,929
Trainable params: 328,929
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50
2231/2231 [==============================] - 3s 1ms/step - loss: 6.7019 - acc: 0.8839
Epoch 2/50
2231/2231 [==============================] - 2s 691us/step - loss: 1.3651 - acc: 0.9364
Epoch 3/50
2231/2231 [==============================] - 1s 659us/step - loss: 0.7879 - acc: 0.9364
Epoch 4/50
2231/2231 [==============================] - 2s 684us/step - loss: 0.5836 - acc: 0.9364
Epoch 5/50
2231/2231 [==============================] - 2s 685us/step - loss: 0.5640 - acc: 0.9364
Epoch 6/50
2231/2231 [==============================] - 2s 775us/step - loss: 0.5502 - acc: 0.9364
Epoch 7/50
2231/2231 [==============================] - 2s 808us/step - loss: 0.5428 - acc: 0.9364
Epoch 8/50
2231/2231 [==============================] - 2s 783us/step - loss: 0.5363 - acc: 0.9364
Epoch 9/50
2231/2231 [==============================] - 2s 770us/step - loss: 0.5262 - acc: 0.9368
Epoch 10/50
2231/2231 [==============================] - 2s 774us/step - loss: 0.5082 - acc: 0.9368 1s - loss: 

In [16]:
# save the model to file
model.save('caption_model_improved.h5')
# save the tokenizer
dump(tokenizer, open('caption_tokenizer_improved.pkl', 'wb'))

In [20]:
sequences[0]

array([ 17, 950, 830, 722,   1,   1,   1,   1,   1,   1])

In [19]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load the model
model = load_model('caption_model_improved.h5')
tokenizer = load(open('caption_tokenizer_improved.pkl', 'rb'))

# load cleaned text sequences
in_filename = 'padded_captions.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

#classification_result should be the first predicted type generated from gerard's classifier
classification_result = ''
#loop through lines and find a caption that begins with the same class
#use that as seed_text

seq_length = 9
seed_text = 'comic_book'
print('seed text: \n')
print(seed_text + '\n')

#seq_length = len(seed_text)

n_words = 10

result = list()
in_text = seed_text
# generate a fixed number of words
for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict_classes(encoded, verbose=0)
    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
        if index == yhat:
            out_word = word
            break
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
    
meme_caption = ' '.join(result)
print('generated caption: \n')
print(meme_caption)

seed text: 

comic_book

generated caption: 

PADDING PADDING PADDING PADDING PADDING PADDING PADDING PADDING PADDING PADDING
